In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import requests
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import nltk.sentiment

from wordcloud import WordCloud
pd.set_option('display.max_colwidth', -1)
import re
from time import strftime

import unicodedata
import json
from pprint import pprint

# Turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

from PIL import Image

import acquire 
import prepare

<ipython-input-1-92e6b57189d7>:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
df = pd.DataFrame(acquire.scrape_github_data(), columns= ['repo', 'language', 'readme_contents'])


In [4]:
df.shape

(600, 3)

In [5]:
#     This function take in a df and the string name for a text column with 
#     option to pass lists for extra_words and exclude_words and
#     returns a df with the text article title, original text, stemmed text,
#     lemmatized text, cleaned, tokenized, & lemmatized text with stopwords removed.

df = prepare.prep_article_data(df)

In [7]:
#backup
df1 = df

In [ ]:
### Breaking down filter_data function located in prepare.py

In [10]:
# 22 null values that will be dropped
df.isnull().sum()

repo               0 
language           22
readme_contents    0 
clean              0 
stemmed            0 
lemmatized         0 
dtype: int64

In [12]:
df = df.dropna()
df.shape[0]

#dropped 22 values

578

In [13]:
def isEnglish(s):
    '''This function filters out for just english words in readme_contents for github repos'''
    try:
        s.encode(encoding='utf-8').decode('ascii')
        
    except UnicodeDecodeError:
        return False
    else:
        return True

In [16]:
# isEnglish function dropped 270 repos

df =df[df.readme_contents.apply(isEnglish) == True]
df.shape[0]

308

In [17]:
df.language.value_counts()

Python               112
JavaScript           55 
Java                 24 
C#                   20 
TypeScript           17 
PHP                  11 
Kotlin               10 
Go                   10 
C++                  7  
Jupyter Notebook     5  
C                    5  
Dart                 4  
Ruby                 4  
HTML                 4  
Swift                3  
Shell                3  
Rust                 2  
CoffeeScript         2  
TeX                  1  
Lua                  1  
OCaml                1  
PowerShell           1  
CSS                  1  
Elixir               1  
Visual Basic .NET    1  
HLSL                 1  
OpenEdge ABL         1  
Elm                  1  
Name: language, dtype: int64

In [18]:
def get_top_5_languages(df):
    '''This function filters for just the top 5 languages and drops the rest'''
    top5_list = list(df.language.value_counts().head(5).index)
    mask = df.language.apply(lambda x: x in top5_list)
    df = df[mask]
    return df

In [20]:
# dropping all values but top 5
# drops 80 repos (May add back if we have time past MVP)
df = get_top_5_languages(df)
df.shape[0]

228

In [21]:
# wrote df to CSV for quicker processing
import os
df.to_csv('NLP.csv')

In [22]:
train, validate, test = prepare.split_data(df)

In [24]:
print ('train ===>', train.shape)
print ('validate ===>', validate.shape)
print('test===>', test.shape)

train ===> (127, 6)
validate ===> (55, 6)
test===> (46, 6)
